In [2]:
import random

# Function to get a custom list of players from the user
def get_player_list():gi
    players = set()
    while True:
        player = input("Enter player name (or 'done' to finish): ").strip().lower()
        if player == 'done':
            if len(players) < 2:
                print("You need at least two players to start the tournament.")
            else:
                break
        elif player in players:
            print(f"{player} is already in the tournament.")
        else:
            players.add(player)
    return players

# Function to assign matches
def assign_matches(available_players, game_counts):
    sorted_players = sorted(available_players, key=lambda p: game_counts[p])
    matches = []
    for i in range(0, min(len(sorted_players), 4), 2):
        matches.append((sorted_players[i], sorted_players[i + 1]))
    return matches

# Function to update leaderboard
def update_leaderboard(leaderboard, winner, loser, score):
    leaderboard[winner]['wins'] += 1
    leaderboard[winner]['points'] += score
    leaderboard[loser]['losses'] += 1

# Function to get valid input for table number
def get_valid_table_number(ongoing_matches):
    while True:
        try:
            finished_table_input = input("\nWhich match finished (enter table number), or 'change' to modify players: ")
            if finished_table_input.lower() == 'change':
                return 'change'
            finished_table = int(finished_table_input) - 1
            if finished_table in ongoing_matches:
                return finished_table
            else:
                print(f"Please enter a valid table number.")
        except ValueError:
            print("Invalid input! Please enter a valid number or 'change'.")

# Function to get valid winner
def get_valid_winner(match):
    while True:
        winner = input(f"Who won the match (enter {match[0]} or {match[1]}): ").strip().lower()
        if winner in match:
            return winner
        else:
            print(f"Invalid input! Please enter either {match[0]} or {match[1]}.")

# Function to get valid score
def get_valid_score():
    while True:
        try:
            score = int(input("Enter the score for the winner: "))
            if score >= 0:
                return score
            else:
                print("Score must be a non-negative number.")
        except ValueError:
            print("Invalid input! Please enter a valid number.")

# Function to modify players
def modify_players(active_players, all_players, leaderboard, game_counts):
    while True:
        action = input("Do you want to 'add' or 'remove' a player? (or 'done' to finish): ").strip().lower()
        if action == 'done':
            break
        elif action == 'add':
            new_player = input("Enter the name of the player to add: ").strip().lower()
            if new_player not in all_players:
                active_players.add(new_player)
                all_players.add(new_player)
                leaderboard[new_player] = {'wins': 0, 'losses': 0, 'points': 0}
                game_counts[new_player] = 0
                print(f"{new_player} has been added to the tournament.")
            else:
                if new_player in active_players:
                    print(f"{new_player} is already active in the tournament.")
                else:
                    active_players.add(new_player)
                    print(f"{new_player} has been reactivated in the tournament.")
        elif action == 'remove':
            player_to_remove = input("Enter the name of the player to remove: ").strip().lower()
            if player_to_remove in active_players:
                active_players.remove(player_to_remove)
                print(f"{player_to_remove} has been removed from active play but remains on the leaderboard.")
            else:
                print(f"{player_to_remove} is not active in the tournament.")
        else:
            print("Invalid input. Please enter 'add', 'remove', or 'done'.")

# Run the tournament
def run_tournament():
    active_players = get_player_list()
    all_players = set(active_players)
    leaderboard = {player: {'wins': 0, 'losses': 0, 'points': 0} for player in all_players}
    game_counts = {player: 0 for player in all_players}
    ongoing_matches = {}

    # Initial match assignment
    initial_matches = assign_matches(list(active_players), game_counts)
    for i, match in enumerate(initial_matches):
        ongoing_matches[i] = match
        game_counts[match[0]] += 1
        game_counts[match[1]] += 1

    while True:
        print("\nOngoing Matches:")
        for table, match in ongoing_matches.items():
            print(f"Table {table + 1}: {match[0]} vs {match[1]}")

        result = get_valid_table_number(ongoing_matches)

        if result == 'change':
            modify_players(active_players, all_players, leaderboard, game_counts)
            continue

        finished_table = result
        finished_match = ongoing_matches[finished_table]

        winner = get_valid_winner(finished_match)
        score = get_valid_score()

        # Determine loser
        loser = finished_match[1] if winner == finished_match[0] else finished_match[0]
        update_leaderboard(leaderboard, winner, loser, score)

        # Remove finished match
        del ongoing_matches[finished_table]

        # Create a list of all active players not currently in a match
        available_players = active_players - set([player for match in ongoing_matches.values() for player in match])

        # If there aren't enough available players, add players with the lowest game count
        while len(available_players) < 2 and len(available_players) < len(active_players):
            least_played = min((p for p in active_players if p not in available_players), key=game_counts.get)
            available_players.add(least_played)

        # Assign new match only if there are enough players
        if len(available_players) >= 2:
            new_matches = assign_matches(list(available_players), game_counts)
            if new_matches:
                ongoing_matches[finished_table] = new_matches[0]  # Only take the first match
                game_counts[new_matches[0][0]] += 1
                game_counts[new_matches[0][1]] += 1

        # Ask if the tournament should continue
        while True:
            continue_tournament = input("\nDo you want to continue the tournament? (yes/no): ").strip().lower()
            if continue_tournament in ['yes', 'no']:
                break
            print("Please enter 'yes' or 'no'.")

        if continue_tournament == 'no':
            break

    # Display final leaderboard and game counts
    print("\nFinal Leaderboard:")
    sorted_leaderboard = sorted(leaderboard.items(), key=lambda item: item[1]['points'], reverse=True)
    for player, stats in sorted_leaderboard:
        print(f"{player}: {stats['wins']} Wins, {stats['losses']} Losses, {stats['points']} Points, {game_counts[player]} Games Played")

if __name__ == "__main__":
    run_tournament()

Enter player name (or 'done' to finish): a
Enter player name (or 'done' to finish): 
Enter player name (or 'done' to finish): 
 is already in the tournament.
Enter player name (or 'done' to finish): b
Enter player name (or 'done' to finish): c
Enter player name (or 'done' to finish): d
Enter player name (or 'done' to finish): e
Enter player name (or 'done' to finish): f
Enter player name (or 'done' to finish): g
Enter player name (or 'done' to finish): h
Enter player name (or 'done' to finish): done

Ongoing Matches:
Table 1:  vs g
Table 2: c vs b

Which match finished (enter table number), or 'change' to modify players: 1
Who won the match (enter  or g):  
Enter the score for the winner: 1

Do you want to continue the tournament? (yes/no): yes

Ongoing Matches:
Table 2: c vs b
Table 1: f vs a

Which match finished (enter table number), or 'change' to modify players: c
Invalid input! Please enter a valid number or 'change'.

Which match finished (enter table number), or 'change' to mod